In [1]:
import pytorch_transformers
import sys
import torch
%load_ext autoreload
%autoreload 2
sys.path.insert(0, "/ssd2/arthur/TREC2019/scripts/")
from msmarco_dataset import MsMarcoDataset
from args_parser import getArgs
from torch.utils.data import DataLoader
import os
from tqdm.autonotebook import tqdm                         
from sklearn.metrics import f1_score, average_precision_score, accuracy_score
import numpy as np

In [2]:
from pytorch_transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("/ssd2/arthur/TREC2019/data/models/distilbert-LNC2/")

In [3]:
data_dir = "/ssd2/arthur/TREC2019/data"

labeled=False
# datasets = ["fulldev", "dev", "test"]
datasets_names = ["LNC2_test"]

batch_size = 1024
dataloaders = {}
datasets = {}

for dataset in datasets_names:
    _file=os.path.join(data_dir, "triples-tokenized", "{}-triples.top100.no_label".format(dataset))
    assert os.path.isfile(_file)
    datasets[dataset] = MsMarcoDataset(_file, data_dir, distil=True, invert_label=True, force=False, labeled=labeled)
    dataloaders[dataset] = DataLoader(datasets[dataset], batch_size=batch_size, shuffle=False)

In [4]:
seed = 42
gpus_to_use = [0, 1, 2, 3, 4, 6, 7]
torch.cuda.manual_seed_all(seed)
n_gpu = torch.cuda.device_count()
device = torch.device("cuda" if (torch.cuda.is_available() and n_gpu > 0) else "cpu")
model = torch.nn.DataParallel(model, device_ids=gpus_to_use)
model = model.to(device)

In [9]:
len(outputs)

1

In [42]:
softmax(torch.as_tensor(_preds))[:,0].cpu().numpy()

array([9.8120734e-05, 2.7500890e-04, 7.0199691e-05, ..., 2.0665339e-06,
       4.0093551e-06, 2.6199716e-06], dtype=float32)

In [44]:
preds[dataset] = softmax(torch.as_tensor(_preds))[:,0].cpu().numpy()

In [17]:
import torch
softmax = torch.nn.Softmax(dim=1)

preds = {}
for dataset in datasets_names:
    eval_loss = 0.0
    nb_eval_steps = 0
    preds[dataset] = None
    out_label_ids = 0
    _preds = None
#     if os.path.isfile(os.path.join(data_dir, 'predictions', '{}_no_label.distilBERT.tensor'.format(dataset))):
#         preds[dataset] = torch.load(os.path.join(data_dir, 'predictions', '{}_distilBERT.tensor'.format(dataset)))
#         preds[dataset] = list(softmax(torch.as_tensor(preds[dataset]))[:, 0].cpu().numpy())
#         continue
    dataloader = dataloaders[dataset]
    for index, batch in tqdm(enumerate(dataloader), desc="{} Dataset".format(dataset), total = len(dataloader)):
        model.eval()
        with torch.no_grad():
            if labeled:
                inputs = {'input_ids': batch[0].to(device),
                          'attention_mask': batch[1].to(device),
                          'labels': batch[3].to(device)}
            else:
                inputs = {'input_ids': batch[0].to(device),
                          'attention_mask': batch[1].to(device)}
            outputs = model(**inputs)
            if labeled:
                tmp_eval_loss, logits = outputs[:2]
                eval_loss+=tmp_eval_loss.mean().item()
                nb_eval_steps+=1
            else:
                logits = outputs[0]

            if _preds is None:
                _preds = logits.detach().cpu().numpy()
                if labeled:
                    out_label_ids = inputs['labels'].detach().cpu().numpy().flatten()

            else:
                batch_predictions = logits.detach().cpu().numpy()
                _preds = np.append(_preds, batch_predictions, axis=0)
                if labeled:
                    batch_ground_truth = inputs['labels'].detach().cpu().numpy().flatten()
                    out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy().flatten(), axis=0)
                    if index%50 == 0:
                        print("\taccuracy: {}".format(accuracy_score(batch_ground_truth, np.argmax(batch_predictions, axis=1))))
                        print("\tf1_score: {}".format(f1_score(batch_ground_truth, np.argmax(batch_predictions, axis=1))))
    
        print(len(_preds))
    if labeled:
        assert len(_preds) == len(out_label_ids)
    torch.save(_preds, os.path.join(data_dir, 'predictions', '{}_no_label.distilBERT.tensor'.format(dataset)))
    preds[dataset] = softmax(torch.as_tensor(_preds))[:,0].cpu().numpy()

1024
2048
3072
4096
5120
6144
7168
8192
9216
10240
11264
12288
13312
14336
15360
16384
17408
18432
19456
20480
21504
22528
23552
24576
25600
26624
27648
28672
29696
30720
31744
32768
33792
34816
35840
36864
37888
38912
39936
40960
41984
43008
44032
45056
46080
47104
48128
49152
50176
51200
52224
53248
54272
55296
56320
57344
58368
59392
60416
61440
62464
63488
64512
65536
66560
67584
68608
69632
70656
71680
72704
73728
74752
75776
76800
77824
78848
79872
80896
81920
82944
83968
84992
86016
87040
88064
89088
90112
91136
92160
93184
94208
95232
96256
97280
98304
99328
100352
101376
102400
103424
104448
105472
106496
107520
108544
109568
110592
111616
112640
113664
114688
115712
116736
117760
118784
119808
120832
121856
122880
123904
124928
125952
126976
128000
129024
130048
131072
132096
133120
134144
135168
136192
137216
138240
139264
140288
141312
142336
143360
144384
145408
146432
147456
148480
149504
150528
151552
152576
153600
154624
155648
155800


In [22]:
datasets

{'LNC2_test': <msmarco_dataset.MsMarcoDataset at 0x7f2921142cc0>}

In [23]:
#load QL scores and normalize

from collections import defaultdict
ql_scores = defaultdict(lambda:defaultdict(lambda:[]))
ordered_topics = defaultdict(lambda:[])
scores_per_topic = defaultdict(lambda:defaultdict(lambda:[]))
for dataset in datasets_names:
    QL_run_file = "/ssd2/arthur/TREC2019/data/runs/LNC2_indri.run".format(dataset)
    last_topic = None
    normalized_scores = []

    with open(QL_run_file, 'r') as inf:
        for counter, line in tqdm(enumerate(inf), desc="reading run file", total=len(datasets[dataset])):
            [topic_id, _, doc_id, _, score, _] = line.split()
            if topic_id not in ordered_topics[dataset]:
                ordered_topics[dataset].append(topic_id)
            scores_per_topic[dataset][topic_id].append((doc_id, score))
    #normalize
    for _id in tqdm(scores_per_topic[dataset], desc="normalizing"):
        _scores = np.asarray([float(x[1]) for x in scores_per_topic[dataset][_id]])
        normalized_scores = (_scores - np.min(_scores))/np.ptp(_scores)
        for (did, _), score in zip(scores_per_topic[dataset][_id], normalized_scores):
            guid = "{}-{}".format(_id, did)
            ql_scores[dataset][guid] = score
print(len(ql_scores["LNC2_test"]))

155800


In [35]:
topic_results

[{'topic_id': '201376',
  'doc_id': 'D238341LNC2',
  'score': array([1., 1.], dtype=float32)},
 {'topic_id': '201376',
  'doc_id': 'D238339LNC2',
  'score': array([0.5343249, 0.5343249], dtype=float32)},
 {'topic_id': '201376',
  'doc_id': 'D2244630LNC2',
  'score': array([0.48666337, 0.48666337], dtype=float32)},
 {'topic_id': '201376',
  'doc_id': 'D353952',
  'score': array([0.46096182, 0.46096182], dtype=float32)},
 {'topic_id': '201376',
  'doc_id': 'D954478LNC2',
  'score': array([0.37722692, 0.37722692], dtype=float32)},
 {'topic_id': '201376',
  'doc_id': 'D238338',
  'score': array([0.30867624, 0.30867624], dtype=float32)},
 {'topic_id': '201376',
  'doc_id': 'D2244630',
  'score': array([0.29501238, 0.29501238], dtype=float32)},
 {'topic_id': '201376',
  'doc_id': 'D1665757LNC2',
  'score': array([0.29168072, 0.29168072], dtype=float32)},
 {'topic_id': '201376',
  'doc_id': 'D2529805',
  'score': array([0.2889043, 0.2889043], dtype=float32)},
 {'topic_id': '201376',
  'doc_id

In [46]:
import subprocess
import os
trec_eval_path = "/ssd2/arthur/trec_eval/trec_eval"
dev_qrel_path = "/ssd2/arthur/TREC2019/data/qrels/dev_qrels"
cmd = "{} -q -c {} {}"
map_cmd = "{} -q -m map {} {}"
best_map = 0.0
runs_format = "{} Q0 {} {} {} DISTILBERT_QL\n" #topic_id, doc_id, ranking, score

alphas = [0.0, 0.85, 1.0]
for dataset in datasets:
#     ql_run_file = os.path.join("/ssd2/arthur/TREC2019/data/runs/{}_QL.run".format(dataset))
    ql_run_file = os.path.join("/ssd2/arthur/TREC2019/data/runs/LNC2_indri.run")
    for alpha in alphas:
        beta = 1-alpha
        out_run_file = os.path.join("/ssd2/arthur/TREC2019/data/runs/{}_distilBert-{}.run".format(dataset, alpha))
        topic_results = [] 
        last_topic = -1
        with open(ql_run_file, 'r') as inf, open(out_run_file, 'w') as outf:
            for counter, (example, score) in enumerate(zip(inf, preds[dataset])):
                topic_id, _, doc_id, _, _, _ = example.split()
                guid = "{}-{}".format(topic_id, doc_id)
                if topic_id != last_topic and len(topic_results) > 0:
                    topic_results.sort(key=lambda x:x['score'], reverse=True)
                    for rank, topic in enumerate(topic_results):
                        outf.write(runs_format.format(topic['topic_id'], topic['doc_id'], rank, topic['score']))
                    topic_results = []
                topic_results.append({'topic_id':topic_id, 'doc_id':doc_id, 'score':alpha*score + beta*ql_scores[dataset][guid]})
                last_topic = topic_id

            #dump last topic
            topic_results.sort(key=lambda x:x['score'], reverse=True)
            for rank, topic in enumerate(topic_results):
                outf.write(runs_format.format(topic['topic_id'], topic['doc_id'], rank, topic['score']))
        
        if labeled:
            qrel_file = os.path.join(data_dir, 'qrels', '{}_qrels'.format(dataset))
            result = subprocess.check_output(cmd.format(trec_eval_path, qrel_file , out_run_file).split()).decode('utf-8')
            maps = subprocess.check_output(map_cmd.format(trec_eval_path, qrel_file, out_run_file).split()).decode('utf-8')
            maps = [float(x.strip().split("\t")[-1]) for x in maps.split("\n") if len(x)>2]
            _map = float(result.split("\n")[-26].split("\t")[-1])
            print("\talpha: {}\t map: {}".format(alpha, _map))
            if _map > best_map:
                best_map = _map
                best_alpha = alpha
